# stage4-1の続き

## ステップ44 パラメータをまとめるレイヤ

### 44.1 Parameterクラスの実装

In [1]:
import numpy as np

from leopard import Variable
from leopard import Parameter
from leopard import Layer

import leopard.functions as F

In [2]:
x = Variable(np.array(1.0))
p = Parameter(np.array(2.0))

y = x * p

In [3]:
print(type(x))
print(type(p))
print(type(y))

<class 'leopard.core.Variable'>
<class 'leopard.core.Parameter'>
<class 'leopard.core.Variable'>


### 44.2 Layerクラスの実装

In [4]:
layer = Layer()

layer.p1 = Parameter(np.array(1))
layer.p2 = Parameter(np.array(2))
layer.p3 = Variable(np.array(3))
layer.p4 = 'test'

print(layer._params)

{'p1', 'p2'}


In [5]:
for name in layer._params:
    print(name, layer.__dict__[name])

p1 variable(1)
p2 variable(2)


### 44.3 Linearクラスの実装
省略

### 44.4 Layerを使ったニューラルネットワークの実装

In [6]:
import leopard.layers as L

In [7]:
# dataset
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

In [8]:
l1 = L.Linear(10) # 出力サイズを指定
l2 = L.Linear(1)

In [9]:
def predict(x):
    y = l1(x)
    y = F.sigmoid(y)
    y = l2(y)
    return y

In [10]:
lr = 0.2
iters = 10000

for i in range(iters):
    y_pred = predict(x)
    loss = F.mean_squared_error(y, y_pred)

    l1.cleargrads()
    l2.cleargrads()
    loss.backward()

    for l in [l1, l2]:
        for p in l.params():
            p.data -= lr * p.grad.data
    if i % 1000 == 0:
        print(loss)

variable(0.8165178492839196)
variable(0.24990280802148895)
variable(0.24609876581126014)
variable(0.2372159081431807)
variable(0.20793216413350174)
variable(0.12311905720649353)
variable(0.07888166506355149)
variable(0.07655073683421637)
variable(0.0763780308623822)
variable(0.07618764131185572)


## ステップ45 レイヤをまとめるレイヤ
Layerクラスに**入れ子**の構造を追加する。つまり、Layerクラスが**別のLayer**を保持できるようにする。

In [11]:
from leopard import Layer

In [12]:
model = Layer()

model.l1 = L.Linear(5) # 出力サイズだけを指定
model.l2 = L.Linear(3)

In [13]:
# 推論を行う関数
def predict(model, x):
    y = model.l1(x)
    y = F.sigmoid(y)
    y = model.l2(y)
    return y

In [14]:
for p in model.params(): # 全てのパラメータにアクセス
    print(p)

variable(None)
variable([0. 0. 0. 0. 0.])
variable(None)
variable([0. 0. 0.])


In [15]:
model.cleargrads()

### 45.2 Modelクラス

In [16]:
from leopard.models import Model

In [17]:
# Model definition
class TwoLayerNet(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.l1 = L.Linear(hidden_size)
        self.l2 = L.Linear(out_size)

    def forward(self, x):
        y = F.sigmoid(self.l1(x))
        y = self.l2(y)
        return y

### 45.3 Modelを使って問題を解く

In [18]:
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

In [19]:
# Hyperparameters
lr = 0.2
max_iter = 10000
hidden_size = 10

In [20]:
model = TwoLayerNet(hidden_size, 1)

In [21]:
for i in range(max_iter):
    y_pred = model(x)
    loss = F.mean_squared_error(y, y_pred)

    model.cleargrads()
    loss.backward()

    for p in model.params():
        p.data -= lr * p.grad.data
    if i % 1000 == 0:
        print(loss)

variable(0.8165178492839196)
variable(0.24990280802148895)
variable(0.24609876581126014)
variable(0.2372159081431807)
variable(0.20793216413350174)
variable(0.12311905720649353)
variable(0.07888166506355149)
variable(0.07655073683421637)
variable(0.0763780308623822)
variable(0.07618764131185572)


### 45.4 MLP(Multi-Layer-Perceptron)クラス
今後のことを見据えて、**N層**のネットワークを作るクラスを実装する。

In [22]:
from leopard.models import MLP

In [23]:
model = MLP((10, 1)) # 2層

In [24]:
for p in model.params():
    print(p.name, p)

W variable(None)
b variable([0.])
W variable(None)
b variable([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])


In [25]:
model = MLP((10, 20, 30, 40, 1)) # 5層

In [26]:
for p in model.params():
    print(p.name, p)

W variable(None)
b variable([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
          0. 0. 0. 0. 0. 0.])
W variable(None)
b variable([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])
W variable(None)
b variable([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
          0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])
W variable(None)
b variable([0. 0. 0. 0. 0. 0. 0. 0. 0. 0.])
W variable(None)
b variable([0.])


## ステップ46　Optimizerによるパラメータ更新

### 46.1　Optimizerクラス
省略

### 46.2　SGDクラスの実装
省略

### 46.3　SGDクラスを使って問題を解く

In [27]:
from leopard import optimizers

In [28]:
lr = 0.2
max_iter = 10000
hidden_size = 10

In [29]:
model = MLP((hidden_size, 1))
optimizer = optimizers.SGD(lr).setup(model)

In [30]:
for i in range(max_iter):
    y_pred = model(x)
    loss = F.mean_squared_error(y, y_pred)

    model.cleargrads()
    loss.backward()

    optimizer.update() # パラメータの更新を一気に行える。
    if i % 1000 == 0:
        print(loss)

variable(1.1827762997377618)
variable(0.24721518059487607)
variable(0.2404155854412563)
variable(0.22093915204680056)
variable(0.14778105835316568)
variable(0.08156167479972622)
variable(0.07643859324328743)
variable(0.07616811473964549)
variable(0.07599896851444594)
variable(0.07584688031253686)


### 46.4 SGD以外の最適化手法
Momentumの実装をしてみる。Momentumは以下の式で表される。
$$

\begin{aligned}

\boldsymbol v_{t+1} & \leftarrow \alpha \boldsymbol v_t - \eta \frac{\partial L}{\partial \boldsymbol W} \\

\boldsymbol W_{t+1} & \leftarrow \boldsymbol W_t + \boldsymbol v_{t+1}

\end{aligned}

$$

通常、$\alpha=0.9, \ \eta = 0.01$を初期値とすることが多い。

## ステップ47 ソフトマックス関数と交差エントロピー誤差

### 47.1 スライス操作のための関数
スライスした場合の逆伝播は、スライスした場所にのみ勾配を加算すれば良い。

In [31]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.get_item(x, 1)
print(y)

variable([4 5 6])


In [32]:
y.backward()
print(x.grad)

variable([[0 0 0]
          [1 1 1]])


In [33]:
indices = np.array([0, 0, 1])
y = F.get_item(x, indices)
print(y)

variable([[1 2 3]
          [1 2 3]
          [4 5 6]])


In [34]:
x.cleargrad()

y = x[1]
y.backward()
print(y)
print(x.grad)

x.cleargrad()

y = x[:, :2]
y.backward()
print(y)
print(x.grad)


variable([4 5 6])
variable([[0 0 0]
          [1 1 1]])
variable([[1 2]
          [4 5]])
variable([[1 1 0]
          [1 1 0]])
